In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [3]:

data_jan = pd.read_parquet("/content/drive/MyDrive/mlops_course/fhv_tripdata_2021-01.parquet")
data_feb = pd.read_parquet("/content/drive/MyDrive/mlops_course/fhv_tripdata_2021-02.parquet")

In [4]:
data_jan.dropOff_datetime

0         2021-01-01 00:44:00
1         2021-01-01 01:07:00
2         2021-01-01 01:51:00
3         2021-01-01 00:21:26
4         2021-01-01 00:53:44
                  ...        
1154107   2021-01-31 23:51:48
1154108   2021-02-01 00:48:03
1154109   2021-01-31 23:29:58
1154110   2021-02-01 00:17:29
1154111   2021-02-01 00:15:00
Name: dropOff_datetime, Length: 1154112, dtype: datetime64[ns]

In [5]:
data_jan_used = data_jan.copy() # data_jan.loc[data_jan.trip_type == 2].copy()

In [6]:
data_jan_used['time'] = data_jan_used.dropOff_datetime - data_jan_used.pickup_datetime
data_jan_used['time_minutes'] = data_jan_used.time.dt.total_seconds() / 60

In [7]:
data_jan_used.drop(data_jan_used.columns.difference(['time_minutes','PUlocationID','DOlocationID']), axis=1, inplace=True)

In [8]:
data_jan_used

,PUlocationID,DOlocationID,time_minutes
0,NaN,NaN,17.000000
1,NaN,NaN,17.000000
2,NaN,NaN,110.000000
3,NaN,72.0,8.283333
4,NaN,61.0,15.216667
...,...,...,...
1154107,7.0,7.0,8.750000
1154108,44.0,91.0,57.600000
1154109,171.0,171.0,16.200000
1154110,15.0,15.0,19.433333


In [9]:
data_jan_used.isnull().sum()/len(data_jan_used)

PUlocationID    0.830307
DOlocationID    0.140558
time_minutes    0.000000
dtype: float64

In [10]:
data_jan_used.fillna(-1, inplace=True)

In [11]:
data_jan_used = data_jan_used.loc[data_jan_used.time_minutes >= 1].loc[data_jan_used.time_minutes <= 60]

In [12]:
data_jan_used

,PUlocationID,DOlocationID,time_minutes
0,-1.0,-1.0,17.000000
1,-1.0,-1.0,17.000000
3,-1.0,72.0,8.283333
4,-1.0,61.0,15.216667
5,-1.0,71.0,9.050000
...,...,...,...
1154107,7.0,7.0,8.750000
1154108,44.0,91.0,57.600000
1154109,171.0,171.0,16.200000
1154110,15.0,15.0,19.433333


In [13]:
categorical = ['PUlocationID','DOlocationID']
numerical =[]

In [14]:
data_jan_used[categorical] = data_jan_used[categorical].astype('str')
data_jan_used

,PUlocationID,DOlocationID,time_minutes
0,-1.0,-1.0,17.000000
1,-1.0,-1.0,17.000000
3,-1.0,72.0,8.283333
4,-1.0,61.0,15.216667
5,-1.0,71.0,9.050000
...,...,...,...
1154107,7.0,7.0,8.750000
1154108,44.0,91.0,57.600000
1154109,171.0,171.0,16.200000
1154110,15.0,15.0,19.433333


In [15]:
train_dict = data_jan_used[categorical].to_dict(orient='records')

In [16]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

In [17]:
len(dv.get_feature_names_out())

525

In [18]:
y_train = data_jan_used.time_minutes

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
print(lr.score(X_train, y_train))

y_pred = lr.predict(X_train)

np.sqrt(mean_squared_error(y_train, y_pred))

0.16927348635052597


10.528519107212679

In [21]:
#categorical = ['PULocationID','DOLocationID']

data_feb_used = data_feb.copy()
data_feb_used['time'] = data_feb_used.dropOff_datetime - data_feb_used.pickup_datetime
data_feb_used['time_minutes'] = data_feb_used.time.dt.total_seconds() / 60

data_feb_used.drop(data_feb_used.columns.difference(['time_minutes','PUlocationID','DOlocationID']), axis=1, inplace=True)

data_feb_used = data_feb_used.loc[data_feb_used.time_minutes >= 1].loc[data_feb_used.time_minutes <= 60]

data_feb_used.fillna(-1, inplace=True)
data_feb_used[categorical] = data_feb_used[categorical].astype('str')
#data_feb_used['PU_DO'] = data_feb_used.PULocationID + data_feb_used.DOLocationID

#categorical = ['PU_DO']
test_dict = data_feb_used[categorical].to_dict(orient='records')

X_test = dv.transform(test_dict)

y_test = data_feb_used.time_minutes

In [22]:
y_pred = lr.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred))

11.014283199431478